- Código para enviar os emails:

In [ ]:
import time
import paho.mqtt.client as mqtt
import smtplib
from email.mime.text import MIMEText

# Configurações do Broker MQTT
broker_address = "endereco_do_broker"
broker_port = 1883
broker_username = "seu_usuario"
broker_password = "sua_senha"

# Configurações do email
email_from = "seu_email@gmail.com"
email_password = "sua_senha"
email_to = "destinatario@gmail.com"
email_subject = "Alerta de Problema no Gateway MQTT"

# Função para enviar o email
def send_email():
    msg = MIMEText("Possível problema no gateway MQTT")
    msg['Subject'] = email_subject
    msg['From'] = email_from
    msg['To'] = email_to

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_from, email_password)
    server.sendmail(email_from, email_to, msg.as_string())
    server.quit()

# Função para lidar com a recepção de mensagens MQTT
def on_message(client, userdata, message):
    # Reinicia o temporizador a cada mensagem recebida
    global last_message_time
    last_message_time = time.time()

# Configura o cliente MQTT
client = mqtt.Client()
client.username_pw_set(broker_username, broker_password)

# Configura as funções de callback
client.on_message = on_message

# Conecta ao Broker MQTT e subscreve ao tópico do gateway
client.connect(broker_address, broker_port)
client.subscribe("topico_do_gateway")

# Define o tempo máximo sem receber mensagens (em segundos)
timeout = 300

# Inicializa o temporizador
last_message_time = time.time()

# Loop principal
client.loop_start()
while True:
    # Verifica se o tempo sem receber mensagens excedeu o limite
    if time.time() - last_message_time > timeout:
        send_email()
        last_message_time = time.time()

    # Espera dez segundos antes de verificar novamente
    time.sleep(10)



- Código escalonável, para vários sensores:

In [ ]:
import time
import paho.mqtt.client as mqtt
import smtplib
from email.mime.text import MIMEText

# Configurações do Broker MQTT
broker_address = "endereco_do_broker"
broker_port = 1883
broker_username = "seu_usuario"
broker_password = "sua_senha"

# Configurações do email
email_from = "seu_email@gmail.com"
email_password = "sua_senha"
email_to = "destinatario@gmail.com"
email_subject = "Alerta de Problema no Gateway MQTT"

# Estrutura de dados para armazenar os detalhes de cada gateway
gateways = [
    {"topic": "topico_gateway1", "timeout": 300},
    {"topic": "topico_gateway2", "timeout": 600},
    {"topic": "topico_gateway3", "timeout": 900}
]

# Função para enviar o email
def send_email():
    msg = MIMEText("Possível problema no gateway MQTT. O tópico"+topic+" faz mais de"+timeout+" segundos, que não envia mensagens.")
    msg['Subject'] = email_subject
    msg['From'] = email_from
    msg['To'] = email_to

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_from, email_password)
    server.sendmail(email_from, email_to, msg.as_string())
    server.quit()

# Função para lidar com a recepção de mensagens MQTT
def on_message(client, userdata, message):
    # Reinicia o temporizador para o gateway correspondente à mensagem recebida
    global last_message_times
    topic = message.topic
    last_message_times[topic] = time.time()

# Configura o cliente MQTT
client = mqtt.Client()
client.username_pw_set(broker_username, broker_password)

# Configura as funções de callback
client.on_message = on_message

# Conecta ao Broker MQTT e subscreve aos tópicos dos gateways
client.connect(broker_address, broker_port)
for gateway in gateways:
    client.subscribe(gateway["topic"])

# Inicializa os temporizadores para cada gateway
last_message_times = {}
for gateway in gateways:
    last_message_times[gateway["topic"]] = time.time()

# Loop principal
client.loop_start()
while True:
    # Verifica cada gateway individualmente
    for gateway in gateways:
        topic = gateway["topic"]
        timeout = gateway["timeout"]

        # Verifica se o tempo sem receber mensagens excedeu o limite para o gateway atual
        if time.time() - last_message_times[topic] > timeout:
            send_email()
            last_message_times[topic] = time.time()  # Reinicia o temporizador

    # Espera dez segundos antes de verificar novamente
    time.sleep(10)


-Código para testar.

In [ ]:
import time
import paho.mqtt.client as mqtt
import smtplib
from email.mime.text import MIMEText

# Configurações do Broker MQTT
broker_address = "10.41.20.15"
broker_port = 1996
broker_username = "piinfra"
broker_password = "piinfra"

# Configurações do email
email_from = "testePI666@gmail.com"
email_password = "eqii mgae gtme aaen"
email_to = "pedrorfrosi@gmail.com"
email_subject = "Alerta de Problema no Gateway MQTT"

# Estrutura de dados para armazenar os detalhes de cada gateway
gateways = [
    {"topic": "gateway/", "timeout": 10}
]

# Função para enviar o email
def send_email():
    msg = MIMEText("Possível problema no gateway MQTT")
    msg['Subject'] = email_subject
    msg['From'] = email_from
    msg['To'] = email_to

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_from, email_password)
    server.sendmail(email_from, email_to, msg.as_string())
    server.quit()

# Função para lidar com a recepção de mensagens MQTT
def on_message(client, userdata, message):
    # Reinicia o temporizador para o gateway correspondente à mensagem recebida
    global last_message_times
    topic = message.topic
    last_message_times[topic] = time.time()

# Configura o cliente MQTT
client = mqtt.Client()
client.username_pw_set(broker_username, broker_password)

# Configura as funções de callback
client.on_message = on_message

# Conecta ao Broker MQTT e subscreve aos tópicos dos gateways
print('Conectando ao MQTT')
client.connect(broker_address, broker_port)
print('Resultado:')
print()
for gateway in gateways:
    client.subscribe(gateway["topic"])

# Inicializa os temporizadores para cada gateway
last_message_times = {}
for gateway in gateways:
    last_message_times[gateway["topic"]] = time.time()

# Loop principal
client.loop_start()
while True:
    # Verifica cada gateway individualmente
    for gateway in gateways:
        topic = gateway["topic"]
        timeout = gateway["timeout"]

        # Verifica se o tempo sem receber mensagens excedeu o limite para o gateway atual
        if time.time() - last_message_times[topic] > timeout:
            send_email()
            last_message_times[topic] = time.time()  # Reinicia o temporizador

    # Espera dez segundos antes de verificar novamente
    time.sleep(5)


-Código para assinar e mostrar as mensagens que são enviadas ao tópico.

In [ ]:
import paho.mqtt.client as mqtt

# Configurações do Broker MQTT
broker_address = "10.41.20.15"
broker_username = "piinfra"
broker_password = "piinfra"
port = 1996
topic = "#"  # Assinando todos os tópicos

# Callback chamada quando o cliente se conecta ao broker
def on_connect(client, userdata, flags, rc):
    print("Conectado ao Broker com código de resultado: " + str(rc))
    # Assina todos os tópicos ao se conectar
    client.subscribe(topic)

# Callback chamada quando uma mensagem é recebida do tópico assinado
def on_message(client, userdata, msg):
    print("Mensagem recebida no tópico {}: {}".format(msg.topic, msg.payload))

# Cria um cliente MQTT
client = mqtt.Client()
client.username_pw_set(broker_username, broker_password)

# Configura as callbacks
client.on_connect = on_connect
client.on_message = on_message

# Conecta-se ao Broker
client.connect(broker_address, port, 60)

# Mantém o cliente em execução
client.loop_forever()
